# Evolution du prix du gazole

##### Auteurs : Mickaël Canhoto (L2 Informatique), Achille Gravouil (L1 Informatique)

*Notre projet consiste à étudier et traiter l'évolution du prix moyen par mois du gazole de 1992 à aujourd'hui.*

### Sommaire

 1. [Introduction](#Introduction)
 2. [Importation des modules](#Importation-des-modules)
 3. [Collecte des données](#Collecte-des-donnees)
 4. [Traitement des données](#Traitement-des-donnees)
 5. [Représentation graphique](#Representation-graphique)
 6. [Conclusion](#Conclusion)

## Introduction

Suite à la situation tendu en Europe et la hausse des prix du gazole, nous avons décidé d'étudier les variations du prix du gazole sur les 30 dernières années. 
Cette étude a pour objectif d'étudier l'évolution du prix afin de crée une représentation graphique et d'étudier celle-ci.

Pour cela nous essaieront de représenter le plus fidélement et le plus explicitement, les données receuillis durant ce projet.
Nous verront donc comment le prix à évolué et nous esssaieront d'anticiper le(s) futur(s) mouvement possible grâce à une fonction

**Pour cela nous nous aiderons des informations recuellis par l'[INSEE](https://www.insee.fr/fr/statistiques/serie/000442588#Tableau)**

## Importation des modules


Nous allons utilisés les modules suivants : 
1. Pandas (importation des données)
2. Bokeh (Création du graphique)
3. statistics (Calcul)

In [2]:
import pandas as pd 
import plotly as pl
import numpy as np
import bokeh as bh
from bokeh.plotting import figure, show
import statistics

## Collecte des données

 * Les données proviennent du site de l'[INSEE](https://www.insee.fr/fr/statistiques/serie/000442588#Tableau). Elle nous présentes le prix moyens mensuels de vente du litre de Gazole en métrople Française de 1992 a 2022.
 

In [3]:
data_test = pd.read_excel ("./data/mesures.xlsx", skiprows=3) #importe les données depuis le fichier excel
data_test = np.flipud(data_test) #Inverse le tableau pour une meilleur visibilité

data_test = pd.DataFrame(data_test, columns=["Période", "Prix"])

with pd.option_context("display.max_rows", 10): #On affiche uniquement 10 lignes
    display(data_test)

/home/while4ent/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Période,Prix
0,1992-01,0.54
1,1992-02,0.54
2,1992-03,0.54
3,1992-04,0.53
4,1992-05,0.54
...,...,...
357,2021-10,1.56
358,2021-11,1.57
359,2021-12,1.54
360,2022-01,1.63


## Traitement des données

* ### Calcule de la moyenne par année
Pour une meilleur lisibilité, nous allons réduire le tableaux par année. Pour cela, nous allons crée une fonction permettant de calculer la moyenne de chaque année et en retourner un tableau

In [45]:
def moyenne_annee():
    
    # fonction "moyenne annee" :
    # Objectif : Calculer la moyenne des prix de chaque année
    # Valeur entrée : aucune
    # Valeur sortie : tableau des moyennes de chaque année
    
    #Initialisation
    data_moy_annee = pd.DataFrame({
        "Période": [],
        "Prix": []}
    )
    r=0
    #Première boucle parcour chaque années
    for i in range (0, 360, 12):
        x=0
        data_moy_annee.loc[r, "Période"] = data_test.loc[i, "Période"]
        
        #Deuxième boucle parcour chaque mois
        for a in range(i,i+12):
            x += data_test.loc[a, "Prix"]
            data_moy_annee.loc[r, "Prix"] = x/12
        r=r+1
        
    #L'année 2022 ne possède uniquement que 2 mois, on réalise alors une boucle appart
    x = 0
    for a in range(2):
        x += data_test.loc[(a + 360), "Prix"]
    
        data_moy_annee.loc[30, "Période"]= data_test.loc[360, "Période"]
        data_moy_annee.loc[30, "Prix"]= x/2
        
    data_moy_annee['Période'] = data_moy_annee['Période'].str[:4]
    return data_moy_annee

#data_moy_horizontal = moyenne_annee().transpose() #Affichage horizontal
#data_moy = moyenne_annee()
#display(data_moy_horizontal)
moyenne_annee().transpose()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
Période,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Prix,0.5375,0.560833,0.595,0.593333,0.6575,0.683333,0.645,0.685833,0.8475,0.799167,...,1.364167,1.301667,1.166667,1.111667,1.233333,1.4375,1.443333,1.273333,1.443333,1.675


* ### Evolution du prix
Nous allons maintenant s'interressé aux années où l'évolution du prix à été forte .

Pour cela nous calculons l'écart type de la moyenne par années puis nous gardons dans un tableau de toutes les années x ou l'écarte type/2 < x - x-1

In [48]:
def Ecart_type(tableau) :
    # fonction "Ecart Type" :
    # Objectif : Calculer l'écart type
    # Valeur entrée : tableau
    # Valeur sortie : écart type du tableau
    list = tableau['Prix'].tolist()
    e_t = statistics.pstdev(list)
    return e_t

# ecart_type = Ecart_type(data_moy)     
#je l'ai rajouté dans la fonction c pas logique de le laisser là si on utilise forte_évolution pour d'autre tableau

def forte_evolution(tableau) :
    # fonction "Forte évolution" :
    # Objectif : Crée un tableau des anneés ou l'évolution du prix a été forte
    # Valeur entrée : tableau
    # Valeur sortie : tableau
    
    k = 0
    Ecart = pd.DataFrame ({
        "Période": [],
        "Prix": []}
    )
    # Ajoute  la ligne i de *tableau dans *Ecart si l'écart entre tableau[i-1] et tableau[i] est supérieur 
    # à *ecart_type/2 (grosse variation)
    for i in range (1, len(tableau)-1) :
        if abs(tableau.loc[i, "Prix"] - tableau.loc[i-1, "Prix"]) > Ecart_type(tableau)/2 : # test si 
            Ecart.loc[k, "Période"] = tableau.loc[i, "Période"]
            Ecart.loc[k, "Prix"] = tableau.loc[i, "Prix"]
            k += 1
    return Ecart

# evolution = forte_evolution(data_moy)
#display(evolution.transpose())
forte_evolution(data_moy).transpose()

,0,1,2,3,4,5,6
Période,2000,2008,2009,2011,2018,2020,2021
Prix,0.8475,1.2825,1.008333,1.345,1.4375,1.273333,1.443333


## Représentation graphique

Pour la représentation graphique, nous avons utilisé la librairie bokeh

In [14]:
#préparation des donneés
x = data_test['Période'].str[:4].tolist()
x2 = data_moy['Période'].tolist()
x3 = evolution['Période'].tolist()
y = data_test['Prix'].tolist()
y2 = data_moy['Prix'].tolist()
y3 = evolution['Prix'].tolist()

#creation des courbes
plot = figure(title="Evolution du prix du Gazole", x_axis_label="Période (En année)", y_axis_label="Prix (En Euro)")
plot.line(x, y, legend_label="Prix par mois", color='grey', line_width=1)
plot.line(x2, y2, legend_label="Prix par années", color='orange', line_width=4)
plot.circle(x3, y3, legend_label="Forte évolution", color='red', size=10, fill_alpha=0.5)

#détais graphique
plot.legend.border_line_alpha = 0.8
plot.legend.location = "top_left"

#montre le résultats

bh.io.output_notebook()
show(plot)

Loading BokehJS ...

In [318]:
#data_test.tail(70).plot(y="Prix", x="Période", kind = "bar")
#plt.rcParams["figure.figsize"] = [100/2.54, 50/2.54]
#plt.show()


In [461]:
#data_test.plot.scatter(y="Prix", x="Période")

In [316]:
#data_moy_annee.plot(y="Prix", x="Période", kind = "bar")
#plt.rcParams["figure.figsize"] = [100/2.54, 50/2.54]
#plt.show()

## Conclusion

 * Conclusions
 * Perspectives